In [1]:
import torch
from torch import nn

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor(
    [[0, 1, 2],
     [3, 4, 5],
     [6, 7, 8]]
)

K = torch.tensor(
    [[0, 1],
     [2, 3]]
)

In [4]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [6]:
model = Conv2D(kernel_size=(2, 2))

In [7]:
model(X)

tensor([[ 6.5622,  9.4972],
        [15.3671, 18.3020]], grad_fn=<AddBackward0>)

## **Object Edge Detection in Images**

In [8]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = torch.tensor([[1.0, -1.0]])

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [11]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [12]:
X.t()  # transpose

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [13]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## **Learning a Kernal**

In [14]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(1, 2), bias=False)

In [15]:
X = X.reshape((1, 1, *X.shape))  # (batch_size, channle, height, width)
Y = Y.reshape(1, 1, *Y.shape)

In [16]:
lr = 0.03

In [17]:
for i in range(20):
    Y_hat = conv2d(X)
    conv2d.zero_grad()  # conv2d에 등록된 parameter들의 grad가 초기화됨
    loss = ((Y_hat - Y)**2).sum()
    loss.backward()
    with torch.no_grad():
        conv2d.weight -= lr * conv2d.weight.grad
    print(f'epoch={i} | loss={loss:.4f}')

epoch=0 | loss=29.4910
epoch=1 | loss=16.7445
epoch=2 | loss=9.8441
epoch=3 | loss=5.9429
epoch=4 | loss=3.6571
epoch=5 | loss=2.2806
epoch=6 | loss=1.4350
epoch=7 | loss=0.9084
epoch=8 | loss=0.5772
epoch=9 | loss=0.3677
epoch=10 | loss=0.2347
epoch=11 | loss=0.1499
epoch=12 | loss=0.0958
epoch=13 | loss=0.0613
epoch=14 | loss=0.0392
epoch=15 | loss=0.0251
epoch=16 | loss=0.0160
epoch=17 | loss=0.0103
epoch=18 | loss=0.0066
epoch=19 | loss=0.0042


In [18]:
conv2d.weight.data

tensor([[[[ 1.0066, -0.9932]]]])